# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [138]:
def priest(sex,age,resp_rate,oxg,heart_beat,bp,temp,alert,insp_oxy,perf):


    """ The function takes the following input parameters:
            Sex (Gender assigned at birth)
            Age in years
            Respiratory rate in breaths per minute
            Oxygen saturation as a percent between 0 and 1
            Heart rate in beats per minute
            Systolic BP in mmHg
            Temperature in degrees C
            Alertness as a string description
            Inspired Oxygen as as string description
            Performance Status as a string description
        It then follows the algorithm provided on the
        MDCalc website to compute a risk percentage and return it as a numeric value between 0 and 1.

        >>> priest('Male',80,20,80,80,120,37,'alert','air','unrestricted normal activity' )
        0.18

        >>> priest('male',24,10,99,60,100,37,'alert','air','unrestricted normal activity' )
        0.03

        >>> priest('female',24,10,99,60,100,36.5,'alert','air','unrestricted normal activity')
        0.02

        >>> priest('male',70,10,99,60,100,38,'confused or not alert','air','unrestricted normal activity')
        0.29

    """
    sex=sex.lower()
    alert=alert.lower()
    perf=perf.lower()
    insp_oxy=insp_oxy.lower()
    score=0

    if sex not in ['male','female'] \
        or alert not in ['alert','confused or not alert'] \
        or insp_oxy not in ['air','supplemental oxygen']\
        or perf not in  ['unrestricted normal activity','limited strenous activity','limited activity, can self-care',\
        'limited strenuous activity, can do light activity','bed/chair bound, no self-care','limited self-care']:
            return None

    else:
        score=0

        if sex=='male':
            score+=1
        if sex =='female':
            score+=0
        if age>=16 and age<=49:
            score+=0
        if age>=50 and age<=65:
            score+=2
        if age>=66 and age<=80:
            score+=3
        if age>80:
            score+=4
        if resp_rate<9:
            score+=3
        if resp_rate>=9 and resp_rate<=11:
            score+=1
        if resp_rate>=12 and resp_rate<=20:
            score+=0
        if resp_rate>=21 and resp_rate<=24:
            score+=2
        if resp_rate>24:
            score+=3

        if oxg>95:
            score+=0
        if oxg>=94 and oxg<=95:
            score+=1
        if oxg>=92 and oxg<=93:
            score+=2
        if oxg<92:
            score+=3
        if heart_beat<41:
            score+=3
        if heart_beat>=41 and heart_beat<=50:
            score +=1

        if heart_beat>=51 and heart_beat<=90:
            score +=0
        if heart_beat>=91 and heart_beat<=110:
            score +=1
        if heart_beat>=111 and heart_beat<=130:
            score +=2
        if heart_beat>130:
            score +=3
        if bp<91:
            score+=3
        if bp>=91 and bp<=100:
            score+=2
        if bp>=101 and bp<=110:
            score+1
        if bp>=111 and bp<=219:
            score+=0
        if bp>219:
            score+3

        if temp<35.1:
            score+=3
        if temp>=35.1 and temp<=36.0:
            score+=1
        if temp>=36.1 and temp<=3.0:
            score+=0

        if temp>=38.1 and temp<=39.0:
            score+=1
        if temp>39.0:
            score+=2
        if alert=='alert':
            score+=0


        if alert=='confused or not alert':
            score+=3
        if insp_oxy=='air':
            score+=0
        if insp_oxy=='supplemental oxygen':
            score+2
        if perf=='unrestricted normal activity':
            score+=0
        if perf=='limited strenuous activity, can do light activity':
            score+=1
        if perf=='limited activity, can self-care':
            score+=2
        if perf=='limited self-care':
            score+=3
        if perf=='bed/chair bound, no self-care':
            score+=4


    #Calculate the probabilities:
    # Example Priest Score

    if score == 0 or score == 1:
        probability = 0.01
    elif score in [2, 3]:
        probability = 0.02
    elif score == 4:
        probability = 0.03
    elif score == 5:
        probability = 0.09
    elif score == 6:
        probability = 0.15
    elif score == 7:
        probability = 0.18
    elif score == 8:
        probability = 0.22
    elif score == 9:
        probability = 0.26
    elif score == 10:
        probability = 0.29
    elif score == 11:
        probability = 0.34
    elif score == 12:
        probability = 0.38
    elif score == 13:
        probability = 0.46
    elif score == 14:
        probability = 0.47
    elif score == 15:
        probability = 0.49
    elif score == 16:
        probability = 0.55
    elif score >= 17 and score <= 25:
        probability = 0.59  #
    else:
        probability = 0.99  # Assuming >99% is approximately 99%

    return probability

In [139]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('Male',80,20,80,80,120,37,'alert','air','unrestricted normal activity' )
Expecting:
    0.18
ok
Trying:
    priest('male',24,10,99,60,100,37,'alert','air','unrestricted normal activity' )
Expecting:
    0.03
ok
Trying:
    priest('female',24,10,99,60,100,36.5,'alert','air','unrestricted normal activity')
Expecting:
    0.02
ok
Trying:
    priest('male',70,10,99,60,100,38,'confused or not alert','air','unrestricted normal activity')
Expecting:
    0.29
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [140]:
import requests

def find_hospital(age, sex, risk):
    """
    Find the hospital to send a patient based on their age, sex, and risk.

    Parameters:
    - age (str): The age of the patient.
    - sex (str): The sex of the patient ('male' or 'female').
    - risk (float): The risk percentage of the patient.

    Returns:
    - str: The name of the hospital to which the patient should be sent, or None if there is an error.

       Example:

    >>> find_hospital("40", "male", 0.1)
    'Southwest Hospital and Medical Center'

    >>> find_hospital("35", "female", 0.05)
    'Select Specialty Hospital - Northeast Atlanta'

    >>> find_hospital("55", "male", 0.3)
    'Southwest Hospital and Medical Center'

    >>> find_hospital("25", "female", 0.15)
    'Select Specialty Hospital - Northeast Atlanta'
    """

    # Make a GET request to the REST API with the provided parameters
    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}"
    response = requests.get(url)

    if response.status_code == 200:
        try:
            # Parse the JSON response
            data = response.json()
            hospital_name = data.get('hospital')
            return hospital_name
        except Exception as e:
            print(f"Error parsing JSON response: {e}")
            return None
    else:
        print(f"Failed to retrieve data from the web service. Status code: {response.status_code}")
        return None

# Test cases
print(find_hospital("40", "male", 0.1))  # Example test case
print(find_hospital("35", "female", 0.05))  # Test case 1
print(find_hospital("55", "male", 0.3))  # Test case 2
print(find_hospital("25", "female", 0.15))  # Test case 3


Southwest Hospital and Medical Center
Select Specialty Hospital - Northeast Atlanta
Southwest Hospital and Medical Center
Select Specialty Hospital - Northeast Atlanta


In [141]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital("40", "male", 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital("35", "female", 0.05)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital("55", "male", 0.3)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital("25", "female", 0.15)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [142]:
import requests

def get_address(hospital_name):
    """
    Get the address of a hospital based on its name.

    Parameters:
    - hospital_name (str): The name of the hospital.

    Returns:
    - str or None: The address of the hospital or None if the hospital is not found.

    Example:
    >>> get_address("COOK MEDICAL CENTER")
    '706 NORTH PARRISH AVENUE'

    >>> get_address("Eastside Medical Center")
    '1700 MEDICAL WAY'

    >>> get_address("DOCTORS HOSPITAL")
    '3651 WHEELER ROAD'

    >>> get_address("Unknown Hospital")

    """
    json_url = 'https://hds5210-data.s3.amazonaws.com/ga_hospitals.json'
    hospital_name = hospital_name.upper()  # Convert the input to uppercase for a case-insensitive search

    try:
        response = requests.get(json_url)
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            hospital = data.get(hospital_name)
            if hospital:
                return hospital.get("ADDRESS")
    except Exception as e:
        print(f"Error: {e}")
    return None

# Test cases
print(get_address("COOK MEDICAL CENTER"))  # Test case 1
print(get_address("Eastside Medical Center"))  # Test case 2
print(get_address("DOCTORS HOSPITAL"))  # Test case 3
print(get_address("Unknown Hospital"))  # Test case 4 (Hospital not found)



706 NORTH PARRISH AVENUE
1700 MEDICAL WAY
3651 WHEELER ROAD
None


In [143]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("COOK MEDICAL CENTER")
Expecting:
    '706 NORTH PARRISH AVENUE'
ok
Trying:
    get_address("Eastside Medical Center")
Expecting:
    '1700 MEDICAL WAY'
ok
Trying:
    get_address("DOCTORS HOSPITAL")
Expecting:
    '3651 WHEELER ROAD'
ok
Trying:
    get_address("Unknown Hospital")
Expecting nothing
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [144]:
import requests
def find_hospital(age, sex, risk):


    # Make a GET request to the REST API with the provided parameters
    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}"
    response = requests.get(url)

    if response.status_code == 200:
        try:
            # Parse the JSON response
            data = response.json()
            hospital_name = data.get('hospital')
            return hospital_name
        except Exception as e:
            print(f"Error parsing JSON response: {e}")
            return None
    else:
        print(f"")
        return None


import requests

def get_address(hospital_name):

    json_url = 'https://hds5210-data.s3.amazonaws.com/ga_hospitals.json'
    hospital_name = hospital_name.upper()  # Convert the input to uppercase for a case-insensitive search

    try:
        response = requests.get(json_url)
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            hospital = data.get(hospital_name)
            if hospital:
                return hospital.get("ADDRESS")

    except Exception as e:
        print(f"Error: {e}")

    return None

def process_people(file_location):

    data = []
    try:
        response = requests.get(file_location)
        if response.status_code == 200:
            lines = response.text.split('\n')
            for line in lines:
                values = line.strip().split('|')
                if len(values) == 11:
                    patient_number, sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status = values
                    # Convert relevant parameters to float

                    hospital_name = find_hospital(age, sex, heart)
                    address = get_address(hospital_name) if hospital_name else hospital_name
                    data.append({
                        patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital_name, address]
                    })
    except Exception as e:
        print(f"")
    return data

# Example usage:
file_location = "https://hds5210-data.s3.amazonaws.com/people.psv"
result = process_people(file_location)
print(result)


[{'patient': ['sex', 'age', 'breath', 'o2 sat', 'heart', 'systolic bp', 'temp', 'alertness', 'inspired', 'status', None, None]}, {'E9559': ['FEMALE', '40', '24', '0.96', '105', '115', '34.9', 'ALERT', 'AIR', 'unrestricted normal activity', 'Emory Dunwoody Medical Center', '4500 NORTH SHALLOWFORD ROAD']}, {'E9385': ['Female', '51', '19', '0.99', '90', '91', '38.1', 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 'Emory Dunwoody Medical Center', '4500 NORTH SHALLOWFORD ROAD']}, {'E3067': ['female', '40', '29', '0.96', '105', '95', '38.1', 'ALERT', 'air', 'unrestricted normal activity', 'Emory Dunwoody Medical Center', '4500 NORTH SHALLOWFORD ROAD']}, {'E9422': ['FEMALE', '66', '19', '0.96', '135', '115', '38.1', 'ALERT', 'AIR', 'bed/chair bound, no self-care', 'Wesley Woods Geriatric Hospital', '1821 CLIFTON ROAD, NE']}, {'E8661': ['MALE', '18', '22', '0.95', '105', '115', '36.3', 'Confused or Not Alert', 'air', 'unrestricted normal activity', 'Emory Dunwoody Medical Cen

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [145]:
import requests
import json

# URLs for the two JSON files to compare
url1 = "https://hds5210-data.s3.amazonaws.com/people_results.json"
url2 = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"

# Function to retrieve JSON data from a URL
def get_json_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Get JSON data from the URLs
json_data1 = get_json_from_url(url1)
json_data2 = get_json_from_url(url2)

# Compare the two JSON objects
if json_data1 == json_data2:
    print("Results Matched")
else:
    print("Results Not Matched")


Results Matched


In [146]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---